# This calculates, summarizes, exports, plots the Fiji image quantification
Uses Fiji generated .txt files as input. Adapted to different protein names.  
Exports results and summarized results as excel file.  
Generates violinplots out of all single cells.

## Initial processing and plotting of individual replicates

In [ ]:
""" open folder and combine all txt files into one dict, which is used subsequently """

import tkinter as tk
from tkinter import filedialog
import pandas as pd
pd.options.mode.copy_on_write = True
import os
import numpy as np
import re

root = tk.Tk()
root.withdraw()

rawfiles = {}
global file_dir
file_dir = filedialog.askdirectory(title="Choose folder with files")
#file_dir = "C:/Users/annal/Dropbox/Quantification_ERreflux"
print(file_dir)
try:
    folder_date = re.findall(r'(\d{6})', file_dir)[1]
except IndexError:
    print("No date")

for filename in sorted(os.listdir(file_dir)):
    print(filename)
    file_path = os.path.join(file_dir, filename)
    if filename.startswith("._"): 
        continue
    elif filename.startswith("~$"):
        continue
    file_name, file_extension = os.path.splitext(filename)
    if file_extension == ".txt":            
        file_data = pd.read_csv(file_path, delimiter="\t", encoding='latin1')
    elif file_extension == ".csv":
        file_data = pd.read_csv(file_path, encoding='latin1')
    else:
        continue
    if "Summary" in file_name:
        continue
    rawfiles[f"{file_name}"] = file_data

root.destroy()
rawfiles.keys()

In [ ]:
import pandas as pd
pd.options.mode.copy_on_write = True
pd.options.display.float_format = '{:,.2f}'.format
import numpy as np
import re
import copy

def Filter (rawfiles):
    """ initial processing: remove columns and combine cell & organelle measurements in one row
    unprocessed txt files have cell and organelle measurements underneath and not assigned to each other yet
    some cells don't have a matching organelle, column names are the same
    fiji thresholding information is on bottom two rows and don't contain measurements
    """
    files = rawfiles.copy()
    regex = r"(?P<POI>^.+) (?P<cond>.+)_\d*" #when name is: POI, condition, replicate
    
    global POI
    ordered_files = {}
    for name, data in files.items():
        POI = re.search(regex, name).group("POI")
        cond = re.search(regex, name).group("cond")
    
    # rename channel names automatically and manually.
        channel_rename = {
            "channel 2": POI,
            "channel 3": POI,
            "channel 1": "Sec63-Scarlet"
        }
    #Drop the bottom rows & other unncessary columns & rename
        data = data.dropna()
        proc_data = data.drop(columns=[' ','MinThr','MaxThr','Message'])
        proc_data.columns = [re.sub(r'^Mean', 'Px Int Mean', col) for col in proc_data.columns]
    #Generate new index based on Label numbers "POI-sfGFP nt_002_cell channel 2:0001-0054"
        num_pattern = r"^.*_(?P<org>\w*) (?P<channel>.*):(?P<num>\d*)-\d*" #match organelle, channel x and number before -
        #go through Label and extract channels and numbers in different columns, generate new index
        proc_data[['Organelles', 'Channel', 'Cell_No']] = proc_data['Label'].str.extract(num_pattern)
        proc_data['Cell_No'] = proc_data['Cell_No'].astype(int)
        proc_data["Channel"] = proc_data["Channel"].map(channel_rename) #rename channels with definded dict POI name
    #Split dfs based on channel and organelle, merge the same channels 
    #aka match each invidual measured cell with its matching organelle
        for channel in proc_data["Channel"].unique():
        # 1. Split/ Filter df by channel
            ch_df = proc_data[proc_data["Channel"] == channel]
            
        # 2. Split df again by organelle and store in dict
    #generate global variables which can be called outside this function and in other functions
            global organelles # a list
            global ref_org # a value of this list
            organelles = ch_df["Organelles"].unique()
            org_df = {org: ch_df[ch_df["Organelles"] == org] for org in organelles}
        # 3. Start with the first organelle
            ref_org = organelles[0]
            for org in organelles:
                if org == ref_org:
            # Rename columns to add organelle
                    organelle_base = (org_df[org]
                                      .drop(columns="Organelles")
                                      .rename(columns=lambda col: f"{col}_{org}" if col not in ["Cell_No", "Label", "Channel"] else col)
                    )
        
        # 4. Merge with all other dfs filtered by organelle
                else:
                    organelle_x = (org_df[org]
                                   .drop(columns="Organelles")
                                   .rename(columns=lambda col: f"{col}_{org}" if col not in ["Cell_No", "Label", "Channel"] else col)
                    )
        # 5. Merge dfs on cells to keep the assignment of a punctum to its cell, tidy up
                    organelle_base = (pd.merge(organelle_base, 
                                              organelle_x.drop(columns=["Label"]), 
                                               on = ["Cell_No", "Channel"],
                                               how="left")
                                      .drop(columns=["Label", "Channel"])
                                     )
        
                    ordered_files[f"{name} {channel}"] = organelle_base
    return ordered_files

def Calculate (filtered):
    """renames columns according to measured organelle, calculates Mean and Standard Error for each seperate file (field of view)
    """
    files = filtered.copy()
    calc_files = {}

    for name, file in files.items():
        summary = pd.DataFrame(columns=[""])
        summary.loc[0] = [None] #dummy row necessary to add new rows
    #Calculate extra statistics
        for org in organelles:
            summary[f"Mean: Area_{org}"] = file[f"Area_{org}"].mean()
            summary[f"Mean: Px Int Mean_{org}"] = file[f"Px Int Mean_{org}"].mean()
            summary[f"SEM: Px Int_{org}"] = file[f"Px Int Mean_{org}"].sem()
    #Add number of cells counted        
        summary["# of cells"] = file["Cell_No"].nunique()     
    #Split file into new dfs puncta and no-puncta
        file_nopuncta = file[file["Px Int Mean_puncta"].isna()]
        file_puncta = file[file["Px Int Mean_puncta"].notna()]
    
    #Store summarized results  
        summary["all puncta"] = file_puncta["Cell_No"].count()
        summary["puncta per cell"] = summary["all puncta"] / summary["# of cells"]
        summary["no puncta per cell"] = file_nopuncta["Cell_No"].count() / summary["# of cells"]
    
    #Count how many times there are puncta per cell
        puncta_counts = file_puncta["Cell_No"].value_counts().value_counts().sort_index()
    
    #Add each count as a new column to summary df
        for puncta_bin, puncta_count in puncta_counts.items():
            col_name = f"{puncta_bin} puncta per cell"
            summary[col_name] = puncta_count / summary["# of cells"]

        final = (file.join(summary.drop(columns=""), how="outer")
             .set_index("Cell_No")
            )
        calc_files[name] = final
    return calc_files

def Summarize (calc_files):
    """
    Combine all single files generated earlier into two summaries of measurements & counted puncta
    Input dfs contain all single cell outputs and a single row with all means and SEM
    The function uses multiindexing to combine it, keep every single df and keep it tidy
    """
    quant_files = copy.deepcopy(calc_files)
    # match file name: Sil1-sfGFP DTT_001 Nsp1_column names
    fileregex = r"(?P<POI>^.*) (?P<cond>.+)_\d* (?P<channel>[a-zA-Z0-9\- ]+)_?(?P<col>.*)" #'?' matches the previous thingy 0 or 1 times
    summary = pd.DataFrame()
            
    for name, file in quant_files.items():
        
        for org in organelles:         
            subset = ["Cell_No", f"Area_{org}", f"Px Int Mean_{org}"]
            puncta_subset = ['# of cells','all puncta'] + [col for col in file.columns if 'per cell' in col] #concat the 2 lists
        
        cond = re.search(fileregex, name).group("cond")
        channel = re.search(fileregex, name).group("channel")
        file.columns = [f"{cond} {channel}_{col}" for col in file.columns] #rename all columns to generate the multiindex
    
        multiindex = pd.MultiIndex.from_arrays(
            arrays=[[channel] * len(file.columns),
                    [cond] * len(file.columns),
                    # file columns look like: DTT channel 2_col_name
                    [col.split("_", 1)[1] for col in file.columns] #split once after the first "_"
                   ], names=["Channel", "Condition", "Col"]
        )
        file.columns = multiindex
        #file_stacked = file.stack(["Channel", "Condition"], future_stack=True).droplevel(0) #drop "Cell_No"
        file_stacked = file.stack(["Channel", "Condition"], future_stack=True).reset_index(level="Cell_No") #put Cell_No back into columns
        puncta_df = file_stacked[puncta_subset]
         
        if summary.empty:
            #summary = file_stacked[subset].dropna()
            summary = file_stacked[subset]
            puncta_summary = puncta_df.dropna()
        else:
            #summary = pd.concat([summary, file_stacked[subset].dropna()], axis=0)
            summary = pd.concat([summary, file_stacked[subset]], axis=0)
            puncta_summary = pd.concat([puncta_summary, puncta_df.dropna()], axis=0)
            
        #subset the df, groupby condition and channel, calculate mean or sum, add a prefix to columns
        summary_mean = summary.loc[:,[col for col in summary.columns]].groupby(["Channel","Condition"]).mean().add_prefix("Mean: ")
        puncta_mean = (puncta_summary
                       .loc[:, [col for col in puncta_summary.columns if col not in ["# of cells", "all puncta"]]] #select all except the specified ones
                       .groupby(["Channel","Condition"]).mean().add_prefix("Mean: ")
                      )
        puncta_sum = puncta_summary.loc[:,["# of cells", "all puncta"]].groupby(["Channel","Condition"]).sum().add_prefix("Combined ")
        
        # Calculate puncta per cell + SEM
        puncta_mean["puncta per cell"] = puncta_sum["Combined all puncta"] / puncta_sum["Combined # of cells"]
        puncta_mean["SEM: puncta per cell"] = summary.groupby(["Channel", "Condition", "Cell_No"]).size().groupby(["Channel", "Condition"]).sem() #calculates group size -> puncta per cell and then again the sem for each group

        # Calculate SEM
        summary_sem = summary.loc[:, [col for col in summary.columns]].groupby(["Channel","Condition"]).sem().add_prefix("SEM: ")                     
                     
        # Merge the mean and sem dfs
        summary_merged = pd.concat([summary_mean, summary_sem], axis=1)
        puncta_merged = pd.concat([puncta_mean, puncta_sum], axis=1)
        
    return summary_merged, puncta_merged

In [ ]:
ordered_files = Filter(rawfiles)
quant_files = Calculate(ordered_files)
summary, puncta = Summarize(quant_files)

# Custom order for Condition & Channel
cond_order = {'nt': 0, 'DTT': 1}
chan_order = {'Nsp1':0, 'Nup116':1, 'Nup159':2, 'Nup84':3, 'Nup157':4, 'Pom152':5}

puncta_order = (
    puncta.reset_index()
      .assign(cond_order=lambda x: x['Condition'].map(cond_order))
      .assign(channel_order=lambda x: x['Channel'].map(chan_order))
      .sort_values(['channel_order', 'cond_order'])
      .drop(columns=['cond_order', 'channel_order'])
      .set_index(['Channel', 'Condition'])
)
summary_order = (
    summary.reset_index()
      .assign(cond_order=lambda x: x['Condition'].map(cond_order))
      .assign(channel_order=lambda x: x['Channel'].map(chan_order))
      .sort_values(['channel_order', 'cond_order'])
      .drop(columns=['cond_order', 'channel_order'])
      .set_index(['Channel', 'Condition'])
)

puncta_order

In [ ]:
"""generate an excel file with all single files as sheets and summary sheets and save"""

import pandas as pd
import numpy as np
import re

save_path = os.path.join(
    os.path.dirname(file_dir),
    f"{os.path.basename(file_dir).split("_",1)[0]}_{POI}"
    )+".xlsx"
print(save_path)

with pd.ExcelWriter(save_path, engine='openpyxl') as writer: #the file is automatically saved when the "with" block is finished
    summary_order.to_excel(writer, sheet_name="CombinedSummary", index=True)
    puncta_order.to_excel(writer, sheet_name="Puncta Summary", index=True)
    #Save each individual merged_df to a combined excel file 
    for name, file in quant_files.items():
        #match the first part of the name without the hyperstack and use as excel sheet name
        sheet_name = name # change according to folder
        file.to_excel(writer, sheet_name=sheet_name, index=True)  


# Plot the results: Histogram

In [ ]:
"""plot bar graph to show distribution of puncta during different conditions and Nups"""

import matplotlib.pyplot as plt

color_palette = ['#000000', '#e6e6e6', 
                 '#8601AF', '#0247FE', '#66B032', '#FCCC1A', '#FB9902', '#FE2712']

fig, ax = plt.subplots(figsize=(12, 5))

histogram = input("Do you only want to plot puncta? -yes -no")

if histogram == "yes":
    puncta_plot = puncta_order.copy().filter(items = ["Mean: puncta per cell"])
    puncta_sem = puncta_order.copy().filter(items = ["SEM: puncta per cell"])
    puncta_plot.columns = puncta_plot.columns.str.replace("Mean: ", "", regex=False)
    puncta_sem.columns = puncta_sem.columns.str.replace("SEM: ", "", regex=False)

    x = np.arange(len(puncta_plot))  # bar groups (12)
    bar_width = 0.9
    n_bars = len(puncta_plot.columns)
    
    # Plot bars with error bars and caps
    for i, col in enumerate(puncta_plot.columns):
        x_pos = x + i * bar_width / n_bars
        ax.bar(
            x_pos,
            puncta_plot[col],
            yerr=puncta_sem[col],
            width=bar_width / n_bars,
            label=col,
            color=color_palette[i],
            edgecolor='black',
            capsize=5
        )
    
    save_path = os.path.join(os.path.dirname(file_dir), f"Puncta count")
else:
    puncta_plot = puncta_order.copy().filter(items = ["Mean: puncta per cell",  "Mean: no puncta per cell", 
                                                      "Mean: 1 puncta per cell", "Mean: 2 puncta per cell", "Mean: 3 puncta per cell", "Mean: 4 puncta per cell", "Mean: 5 puncta per cell", "Mean: 6 puncta per cell"
                                                     ])
    puncta_plot.columns = puncta_plot.columns.str.replace("Mean: ", "", regex=False)
    
    puncta_plot.plot(kind='bar', ax=ax, width=0.9, color=color_palette, edgecolor='black')
    
    save_path = os.path.join(os.path.dirname(file_dir), f"Puncta Histogram")


# Set axis titles
ax.set_title("Frequency of cytosolic Nup puncta per cell")
ax.set_ylabel("Puncta Count per cell", fontsize=12)
ax.set_xlabel("")

# Generate stacked x-axis labels
# Two-line xtick labels: Condition (bottom) + Channel (top)
ax.set_xticklabels([f"{cond}" for chan, cond in puncta_plot.index], 
                   rotation=0, fontsize=10)
ax.set_xticks(x -0.4 + bar_width / 2)
ax.tick_params(axis='x', length=0)  # "remove" xticks

# Add a top x-axis for 'Channel' (e.g., Nsp1)
channels = [chan for chan, cond in puncta_plot.index]

# Find where each channel starts and ends
positions = np.arange(len(channels))
channel_labels = []
channel_centers = []
for ch in dict.fromkeys(channels):  # preserves order & uniqueness
    idx = [i for i, c in enumerate(channels) if c == ch]
    channel_centers.append((sum(idx) / len(idx)))
    channel_labels.append(ch)

# Add secondary x-axis on top
ax2 = ax.secondary_xaxis('bottom')
ax2.set_xticks(channel_centers)
ax2.set_xticklabels(channel_labels, fontsize=12)
ax2.tick_params(axis='x', length=0, pad=20) #remove xticks, adjust label spacing


# Legend outside
ax.legend(
    title='Mean counts per cell',
    bbox_to_anchor=(1.01, 1),
    loc='upper left',
    borderaxespad=0.
)

# save the plots
saveplots = input("Do you want to save the plots? -yes -no")   

if saveplots == "yes":
    plt.savefig((save_path +".svg"), 
                format='svg', dpi=300, bbox_inches='tight')  # Save with high resolution, crop whitespace around
    plt.savefig((save_path +".png"), 
                dpi=300, bbox_inches='tight')  # Save with high resolution, crop whitespace around


plt.tight_layout()
plt.show()
